<h4>Writing like Shakespeare</h4>

In [67]:
import io
import sys

import numpy as np

import tensorflow as tf
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, Masking
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import get_file
from tensorflow.keras.utils import pad_sequences

In [68]:
def build_data(text, Tx=40, stride=3):
    # input data will be the tokens from the real poems
    X = []
    # labels will be the next character that should come
    Y = []
    for i in range(0, len(text)-Tx, stride):
        X.append(text[i : i + Tx])
        Y.append(text[i + Tx])
    return X, Y


def on_epoch_end(epoch, logs):
    None

def vectorization(X, Y, n_x, char_indices, Tx=40):
    m = len(X)
    x = np.zeros((m, Tx, n_x), dtype=bool)
    y = np.zeros((m, Tx), dtype=bool)

    for i, sentence in enumerate(X):
        for j, char in enumerate(sentence):
            x[i, j, char_indices[char]] = 1
        y[i, char_indices[Y[i]]] = 1

    return x, y

In [69]:
print('Loading text data...')
text = open("shakespeare.txt", "r").read().lower()
Tx = 40
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print("Creating training set...")
X, Y = build_data(text, Tx, stride=3)

print('Vectorizing training set...')
x, y = vectorization(X, Y, n_x=len(chars), char_indices=char_indices)
model = load_model('models/model_shakespeare_kiank_350_epoch.h5')

Loading text data...
Creating training set...
Vectorizing training set...


In [70]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y, batch_size=64, epochs=1, callbacks=[print_callback])

ValueError: in user code:

    File "/home/str/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/home/str/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/str/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/home/str/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/home/str/.local/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/str/.local/lib/python3.10/site-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        r